In [1]:
from jetcam.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import cv2
import numpy as np
from jetracer.nvidia_racecar import NvidiaRacecar
camera = CSICamera(width=600, height=600)
car= NvidiaRacecar()
image = camera.read()

image_widget = ipywidgets.Image(format='jpeg')

image_widget.value = bgr8_to_jpeg(image)
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [2]:


kernel_size=5
low_threshold=50
high_threshold=200
rho=2
theta=np.pi/180
threshold=90
min_line_len=120
max_line_gap=150
a=0.8
b=1
r=0
thickness=5
def draw_lines(image,lines, color=[255,0,0],thickness=5):
        if lines is not None :
            for line in lines:
                for x1,y1,x2,y2 in line:
                    cv2.line(image,(x1, y1), (x2, y2), color, thickness)
                
            
            #return image
def hough_lines(image,rho, theta, threshold, min_line_len, max_line_gap):
    lines=cv2.HoughLinesP(image,rho,theta,threshold,np.array([]), minLineLength=min_line_len,maxLineGap=max_line_gap)
    line_img=np.zeros((image.shape[0], image.shape[1], 3),dtype=np.uint8)
    draw_lines(line_img,lines)
    
    return line_img
#masked function
def region_of_interest(image,vertices):
    mask= np.zeros_like(image)
    if len(image.shape) >2:
        channel_count = image.shape[2]
        ignore_mask_color =(255,)*channel_count
    else:
        ignore_mask_color=255

    cv2.fillPoly(mask,vertices, ignore_mask_color)
    masked_image=cv2.bitwise_and(image,mask)
    return masked_image





camera.running = True
#coding
def update_image(change):
    image = change['new']
    
    
    #이 밑에부터 차선 코딩
    image_gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)# gray process
    image_gaussian= cv2.GaussianBlur(image_gray,(kernel_size,kernel_size),0)# kernelsize 입력
    image_canny=cv2.Canny(image_gaussian,low_threshold,high_threshold)#edge detect
    imshape=image.shape
    vertices= np.array([[(20,imshape[0]),(250,250),(350,250),(imshape[1]-20,imshape[0])]],dtype=np.int32)
    mask=region_of_interest(image_canny,vertices)
    lines=hough_lines(mask,rho, theta, threshold, min_line_len, max_line_gap)
    x_left=300
    x_right=300
    radius = 5
    x_left = np.round(x_left).astype("int")
    x_right = np.round(x_right).astype("int")
    radius = np.round(radius).astype("int")
    
    y=400
    while (x_left>0):
        x_left=x_left-3
        if ((x_left,y)==[255,0,0]):
            break
    while (x_right<600):
        x_right=x_right+3
        if((x_right,y)==[255,0,0]):
            break
    c=(x_left+x_right)/2
    c=np.round(c).astype("int")
    load_center=((x_left+x_right)/2,y)
    camera_center=(300,400)
    #load_center=[(x_right+x_left)/2,y] #load center
   # camera_center=[224,y] #앞이 x라고 가정 (반대였던거같기도)
    #이 점들을 어느 image에 그릴지 정해야됨.
    color_camera_center=(255,255,0)# rbg sequence is bgr
    color_load_center=(255,255,255)
    
    thickness=-1
    
    cv2.circle(lines,(300, 400),3,(255,255,0),-1)#mark the point in the center of the camera yellow
    #cv2.circle(lines,(c,400), 3,(255,255,255),-1)#mark the point in the center of load white
    
    lines_edges=cv2.addWeighted(image,a,lines,b,r)
    seta=arctan((load_center-camera_center)/(600-y)
    car.steering = 0.3
    
    image_widget.value = bgr8_to_jpeg(lines_edges)
    
camera.observe(update_image, names='value')


SyntaxError: invalid syntax (<ipython-input-2-c30d4bd6022f>, line 91)